# Projeto

## Objetivo
O objetivo deste projeto é realizar web scraping do Yahoo Finance para coletar informações financeiras e criar um banco de dados sob demanda. Posteriormente, planeja-se expandir o projeto criando uma API para hospedagem.

## Etapa 1: Web Scrapping

In [5]:
import requests
from bs4 import BeautifulSoup
import re
import datetime
import sqlite3
from flask import Flask, jsonify, request

In [205]:
def check_symbol_existence(symbol):
    url = f"https://query1.finance.yahoo.com/v8/finance/chart/{symbol}?range=1d"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  

        exist = 'No data found, symbol may be delisted' not in response.text
        if not exist:
            print(f"A ação '{symbol}' não existe ou não foi encontrada.")
            return False
        
        metrics = {}
        metrics['Symbol'] = re.findall(r'"symbol":"(.*?)"', response.text)[0]
        metrics['type'] = re.findall(r'"instrumentType":"(.*?)"', response.text)[0]
        return metrics

    except requests.exceptions.RequestException as e:
        print(f"Erro na request de teste, A ação '{symbol}' não existe ou não foi encontrada.")
        return False


A função "check_symbol_existence" se fez necessária uma vez que a consulta feita em "get_stock_metrics" eventualmente gerava uma resposta "falsa" ao realizar a consulta se símbolos que o Yahoo Finance detectava como "parecidos" com símbolos existentes, gerando uma resposta referente à listagem de "Symbols similar to ..."

<img src="imgs/symbolsSimilarTo.png" alt="Página Symbols similar to ..." height="200" width="400">
Na implementação da função, se a request for bem-sucedida e obtivermos informações relevantes sobre o ativo, já registramos o marcador (Symbol) e o tipo (type) do ativo no dicionário que conterá informações do ativo. Para obter os respectivos valores, utilizamos REGEX.

<img src="imgs/check_symbol_existence_response.png" alt="Página da request de check_symbol_existence" height="500">

In [206]:
def get_stock_metrics(symbol):
    symbol = symbol.upper()
    url = f"https://finance.yahoo.com/quote/{symbol}/key-statistics"
    headers = {"User-Agent": "https://query1.finance.yahoo.com/v8/finance/chart/NVDA?region=US&lang=en-US&includePrePost=false&interval=2m&useYfid=true&range=1d&corsDomain=finance.yahoo.com&.tsrc=finance"}

    try:
        print(f'Checando a existencia de {symbol}...')
        
        metrics = check_symbol_existence(symbol)
        if metrics is False:  
            return None         
        print('Encontrado! \nAguarde ...')
        response = requests.get(url, headers=headers)

        soup = BeautifulSoup(response.text, 'html.parser')
        
        metrics['qsp-price'] = soup.find('fin-streamer', attrs={'data-test': 'qsp-price'})['value']
        
        table_rows = soup.find_all('tr')
        
        for row in table_rows:
            cells = row.find_all('td')

            if len(cells) == 2:
                metric = cells[0].text.strip()
                value = cells[1].text.strip()
                metrics[metric] = value
        
        metrics['timestamp'] = datetime.datetime.now().isoformat()
        print('\n')
        print('Metricas Obtidas!')
        return metrics
    
    except requests.exceptions.HTTPError as e:
        print("Erro na request:", e)
        print("A ação foi encontrada porém, não foi possível obter as estatisticas da ação.")
        return None

Para a implementação de get_stock_metrics, utilizei a biblioteca BeautifulSoup. Ao acessar uma página da aba "Statistics", percebemos que, separadas do valor principal, as informações estão representadas em tabelas, cada uma relacionada a um tópico:

<img src="imgs/key-statistics_page.png" alt="Página Symbols similar to ..." height="200" width="400">

Logo, para obter o valor principal, basta realizar o parse do HTML obtido na request e pegar o valor relacionado ao marcador "regularMarketPrice". Para as outras informações, será necessário outra saída.

Inspecionando a página, percebe-se uma maneira de explorar o padrão no HTML para obter os dados. Cada métrica está dentro de uma tag < tr >:

<img src="imgs/metrics.png" alt="Padrão de organização das métricas" height="200" width="300">

Dentro de cada tag < tr >, temos duas tags < td >, a primeira se refere ao tipo da métrica e a segunda ao valor respectivo dessa métrica.

<img src="imgs/metricsValue.png" alt="Padrão de organização dos valores das métricas" height="200" width="300">

Dessa forma, torna-se simples obtermos as métricas. Utilizamos o BeautifulSoup para obter todas as linhas que contêm tags < tr >. Então, percorremos a linha; ao encontrar a primeira tag < td >, armazenamos o valor dela e fazemos o mesmo para a segunda. Esses valores serão associados entre si como chave e valor no dicionário de métricas.

In [211]:
symbol = input("Digite o identificador do ativo: ")

    
metrics = get_stock_metrics(symbol)
if metrics is not None:
    for k, v in metrics.items():
        print(f"{k}: {v}")



Digite o identificador do ativo: amzn
Checando a existencia de AMZN...
Encontrado! 
Aguarde ...


Metricas Obtidas!
Symbol: AMZN
type: EQUITY
value: 5178.51
Market Cap (intraday): 1.81T
Enterprise Value: 1.86T
Trailing P/E: 60.17
Forward P/E: 41.32
PEG Ratio (5 yr expected): 2.36
Price/Sales (ttm): 3.18
Price/Book (mrq): 8.98
Enterprise Value/Revenue: 3.24
Enterprise Value/EBITDA: 20.82
Beta (5Y Monthly): 1.17
52-Week Change 3: 73.42%
S&P500 52-Week Change 3: 28.64%
52 Week High 3: 180.14
52 Week Low 3: 96.29
50-Day Moving Average 3: 166.20
200-Day Moving Average 3: 143.40
Avg Vol (3 month) 3: 44.86M
Avg Vol (10 day) 3: 38.41M
Shares Outstanding 5: 10.39B
Implied Shares Outstanding 6: 10.39B
Float 8: 9.22B
% Held by Insiders 1: 9.19%
% Held by Institutions 1: 62.84%
Shares Short (Feb 29, 2024) 4: 66.87M
Short Ratio (Feb 29, 2024) 4: 1.3
Short % of Float (Feb 29, 2024) 4: 0.85%
Short % of Shares Outstanding (Feb 29, 2024) 4: 0.64%
Shares Short (prior month Jan 31, 2024) 4: 75.18M
Forwar

# Etapa 2: Criação da database

In [196]:
def adjust_metrics_keys (metrics):
    if metrics is not None:
        new_data_dict = {}
        for key, value in metrics.items():
            
            # Remove conteudo entre parenteses do que sera uma coluna
            new_key = key.split('(')[0].strip()
            if new_key != "Symbol":
                new_key = new_key.lower()
            
            # Substituindo valores invalidos para nome de coluna
            new_key = new_key.replace('%', 'percent')
            new_key = re.sub(r'\W+', '_', new_key)
            
            # Verificar se a chave não começa com um número
            if not new_key[0].isdigit():
                new_data_dict[new_key] = value
                # Alterar tipo INDEX para evitar conflito no SQLite
                if value == "INDEX":
                    new_data_dict[new_key] = "CURRENCY"
        # Atualizar o dicionário original
        metrics.clear()  # Limpar o dicionário original
        metrics.update(new_data_dict)  # Atualizar o dicionário original com as chaves atualizadas



In [ ]:
def create_table(cursor, table_name, columns):
    
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name=?", (table_name,))
    existing_table = cursor.fetchone()
    
    if not existing_table:
        cursor.execute(f"CREATE TABLE IF NOT EXISTS {table_name} (Symbol TEXT PRIMARY KEY, {', '.join(columns)})")
    
    cursor.execute(f"PRAGMA table_info({table_name})")
    existing_columns = [column[1] for column in cursor.fetchall()]
    # Adicionar as colunas ausentes
    for column in columns:
        if column not in existing_columns:
            cursor.execute(f"ALTER TABLE {table_name} ADD COLUMN {column} REAL")

In [201]:
def insert_data(cursor, table_name, data):
    cursor.execute(f"PRAGMA table_info({table_name})")
    table_columns = [column[1] for column in cursor.fetchall()]

    for key in data.keys():
        if key not in table_columns and key != 'type' and key != 'Symbol':
            raise ValueError(f"A coluna '{key}' não existe na tabela '{table_name}'")

    columns = ', '.join([col for col in data.keys() if col != 'type'])
    
    values = tuple(data[key] for key in data.keys() if key != 'type')

    placeholders = ', '.join(['?' for _ in range(len(values))])

    cursor.execute(f"DELETE FROM {table_name} WHERE Symbol = ?", (data['Symbol'],))

    cursor.execute(f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})", values)


In [202]:
symbol = input("Digite o símbolo da empresa (por exemplo, 'AAPL' para Apple Inc.): ")

    
metrics = get_stock_metrics(symbol.upper())

Digite o símbolo da empresa (por exemplo, 'AAPL' para Apple Inc.): spy
Checando a existencia de SPY...
Encontrado! 
Aguarde ...


Metricas Obtidas!


In [2]:
# Adapta o nome das chaves no dicionario, para nomes validos de coluna 
adjust_metrics_keys(metrics)

for k, v in metrics.items():
        print(f"{k}: {v}")

        
# Conectar ao banco de dados
conn = sqlite3.connect('finance_data.db')
cursor = conn.cursor()

# Cria/Atualiza tabela com base nas metricas coletadas
table_name = metrics['type']
create_table(cursor, table_name, [key for key in metrics.keys() if key != 'type' and key != 'Symbol'])

# Insere/Atualiza os dados do ativo na tabela destinada ao seu tipo
insert_data(cursor, table_name, metrics)

# Commit e fechar conexão
conn.commit()
conn.close()

NameError: name 'adjust_metrics_keys' is not defined

## Etapa 3: Criação da API

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with watchdog (windowsapi)


SystemExit: 1

In [10]:
import os

def delete_database(database_name):
    try:
        # Verificar se o arquivo do banco de dados existe
        if os.path.exists(database_name):
            # Excluir o arquivo do banco de dados
            os.remove(database_name)
            print(f"O banco de dados '{database_name}' foi excluído com sucesso.")
        else:
            print(f"O banco de dados '{database_name}' não foi encontrado.")
    except Exception as e:
        print(f"Ocorreu um erro ao excluir o banco de dados '{database_name}': {e}")

# Chame a função delete_database com o nome do banco de dados que deseja excluir
delete_database('finance_data.db')


O banco de dados 'finance_data.db' foi excluído com sucesso.
